<a href="https://colab.research.google.com/github/yeesem/Time-Series-Data-Analysis/blob/main/Stock_data_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime

In [4]:
data = yf.download('1155.KL',start = '2018-01-01',end = '2021-01-01')

[*********************100%%**********************]  1 of 1 completed


In [5]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,9.65,9.68,9.59,9.66,6.455371,5284100
2018-01-03,9.66,9.78,9.66,9.78,6.535563,13946300
2018-01-04,9.78,9.82,9.78,9.80,6.548926,18711500
2018-01-05,9.80,9.82,9.79,9.80,6.548926,17266600
2018-01-08,9.80,9.85,9.79,9.85,6.582339,19238800


In [6]:
data_range = pd.date_range(data.index[0],data.index[-1])

In [8]:
df = pd.DataFrame(index=data_range)
df.head()

""
2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-06


In [10]:
#The original data does not include weekend
df = df.join(data,how = 'outer')
df.head()

,Open,High,Low,Close,Adj Close,Volume
2018-01-02,9.65,9.68,9.59,9.66,6.455371,5284100.0
2018-01-03,9.66,9.78,9.66,9.78,6.535563,13946300.0
2018-01-04,9.78,9.82,9.78,9.80,6.548926,18711500.0
2018-01-05,9.80,9.82,9.79,9.80,6.548926,17266600.0
2018-01-06,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
cols = df.columns.drop(['Volume'])
df[cols] = df[cols].fillna(method='ffill')
df['Volume'] = df['Volume'].fillna(0)
df.head(10)

,Open,High,Low,Close,Adj Close,Volume
2018-01-02,9.65,9.68,9.59,9.66,6.455371,5284100.0
2018-01-03,9.66,9.78,9.66,9.78,6.535563,13946300.0
2018-01-04,9.78,9.82,9.78,9.80,6.548926,18711500.0
2018-01-05,9.80,9.82,9.79,9.80,6.548926,17266600.0
2018-01-06,9.80,9.82,9.79,9.80,6.548926,0.0
2018-01-07,9.80,9.82,9.79,9.80,6.548926,0.0
2018-01-08,9.80,9.85,9.79,9.85,6.582339,19238800.0
2018-01-09,9.82,9.85,9.80,9.83,6.568975,14892600.0
2018-01-10,9.81,9.85,9.81,9.84,6.575657,18220300.0
2018-01-11,9.84,9.85,9.83,9.84,6.575657,11074400.0


In [16]:
#Save the file as csv file
df.to_csv('daily_price_full.csv')

In [17]:
!head daily_price_full.csv

,Open,High,Low,Close,Adj Close,Volume
2018-01-02,9.649999618530273,9.680000305175781,9.59000015258789,9.65999984741211,6.455371379852295,5284100.0
2018-01-03,9.65999984741211,9.779999732971191,9.65999984741211,9.779999732971191,6.535562515258789,13946300.0
2018-01-04,9.779999732971191,9.819999694824219,9.779999732971191,9.800000190734863,6.54892635345459,18711500.0
2018-01-05,9.800000190734863,9.819999694824219,9.789999961853027,9.800000190734863,6.54892635345459,17266600.0
2018-01-06,9.800000190734863,9.819999694824219,9.789999961853027,9.800000190734863,6.54892635345459,0.0
2018-01-07,9.800000190734863,9.819999694824219,9.789999961853027,9.800000190734863,6.54892635345459,0.0
2018-01-08,9.800000190734863,9.850000381469727,9.789999961853027,9.850000381469727,6.582339286804199,19238800.0
2018-01-09,9.819999694824219,9.850000381469727,9.800000190734863,9.829999923706055,6.56897497177124,14892600.0
2018-01-10,9.8100004196167,9.850000381469727,9.8100004196167,9.84000015258789,6.5756573677

In [18]:
#AWS Forecast requires a column called item_id
df['item_id'] = 'MAYBANK'

In [19]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,item_id
2018-01-02,9.65,9.68,9.59,9.66,6.455371,5284100.0,MAYBANK
2018-01-03,9.66,9.78,9.66,9.78,6.535563,13946300.0,MAYBANK
2018-01-04,9.78,9.82,9.78,9.80,6.548926,18711500.0,MAYBANK
2018-01-05,9.80,9.82,9.79,9.80,6.548926,17266600.0,MAYBANK
2018-01-06,9.80,9.82,9.79,9.80,6.548926,0.0,MAYBANK


In [20]:
#Leave the last 30 points forecast comparison
FORECAST_LENGTH = 30
train = df.iloc[:-FORECAST_LENGTH]

In [21]:
#AWS dofferentiates between 'target time series' and 'related time series'
#'target time series'  - output
#'related time series' - features
train_target_series = train[['Close','item_id']]
train_related_series = train[train.columns.drop(['Close'])]

In [22]:
#Save the data which we will upload to s3 later
train_target_series.to_csv('daily_price_target_series.csv')
train_related_series.to_csv('daily_price_related_series.csv')

In [23]:
!head -n 5 daily_price_target_series.csv

,Close,item_id
2018-01-02,9.65999984741211,MAYBANK
2018-01-03,9.779999732971191,MAYBANK
2018-01-04,9.800000190734863,MAYBANK
2018-01-05,9.800000190734863,MAYBANK


In [24]:
!head -n daily_price_related_series.csv

head: invalid number of lines: ‘daily_price_related_series.csv’
